In [20]:
import pandas as pd
import numpy as np
import pickle
import random
import os
import math

In [21]:
root = '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/'

'''
Context: 
group_id,day_of_week,time_of_day,location,weather,previous_step_count,last_step_count,variation_in_step_counts
'''

In [22]:
with open('{}steps_both_groups_logs_no_dosage.pkl'.format(root),'rb') as f:
    dists = pickle.load(f)

In [23]:
with open('{}interventions_both_groups.pkl'.format(root),'rb') as f:
    intervention_dists = pickle.load(f)

In [24]:
def get_location_prior(group_id,day_of_week,time_of_day):
    with open('{}initial_location_distributions.pkl'.format(root),'rb') as f:
        loc_lookup = pickle.load(f)
    key = '{}-{}-{}'.format(group_id,day_of_week,time_of_day)
    
    ##make a bit smoother while loop instead 
    if key in loc_lookup:
        ps = loc_lookup[key]
    else:
        key =  '{}-{}'.format(group_id,day_of_week)
        if key  in loc_lookup:
            ps = loc_lookup[key]
        else:
            key =  '{}'.format(group_id)
            if key  in loc_lookup:
                ps = loc_lookup[key]
                
            else:
                ps = loc_lookup['mean']
                
    val = np.argmax(np.random.multinomial(100,ps))
    return val

In [25]:
def get_weather_prior(group_id,day_of_week,time_of_day):
    with open('{}initial_weather_distributions.pkl'.format(root),'rb') as f:
        loc_lookup = pickle.load(f)
    key = '{}-{}-{}'.format(group_id,day_of_week,time_of_day)
    
    ##make a bit smoother while loop instead 
    if key in loc_lookup:
        ps = loc_lookup[key]
    else:
        key =  '{}-{}'.format(group_id,day_of_week)
        if key  in loc_lookup:
            ps = loc_lookup[key]
        else:
            key =  '{}'.format(group_id)
            if key  in loc_lookup:
                ps = loc_lookup[key]
                
            else:
                ps = loc_lookup['mean']
                
    val = np.argmax(np.random.multinomial(100,ps))
    return val

In [26]:
def get_time_of_day(an_index):
    with open('{}hour_to_id.pkl'.format(root),'rb') as f:
        hour_lookup = pickle.load(f)
    return hour_lookup[an_index.hour]
    

In [27]:
def get_day_of_week(an_index):
    with open('{}day_to_id.pkl'.format(root),'rb') as f:
        hour_lookup = pickle.load(f)
    return hour_lookup[an_index.dayofweek]
    

In [57]:
def get_index(key):
    
    keys = ['group_id','day_of_week','time_of_day','dosage','location','weather','pretreatment','yesterday','variation']
    
    kl = {keys[i]:i for i in range(len(keys))}
    
    return kl[key]

In [30]:
def get_initial_context(num_people,first_index):
    '''States:
    [group,day_of_week,time_of_day,location,weather,dosage,previous_step_count]
    
    '''
    
    all_people = []
    for person in range(num_people):
        group_id = int(random.random()>.95)+1
        #group_id = 2
        day_of_week = get_time_of_day(first_index)
        time_of_day = get_day_of_week(first_index)
        first_location = get_location_prior(group_id,day_of_week,time_of_day)
        weather = get_weather_prior(group_id,day_of_week,time_of_day)
        #weather = 0 
        dosage = 1
        variation = 1
        pretreatment = 0 
        ysc = 0
        #day_of_week,time_of_day,dosage,location,last_steps,last_steps_hour,varia
        all_people.append([group_id,day_of_week,time_of_day,dosage,first_location,weather,pretreatment,ysc,variation])
        
    return all_people

In [31]:
def get_initial_steps(contexts):
    
    return [steps_given_context(person_context) for person_context in contexts]

    

In [32]:
def get_time(current_time):
    
    #needs to be a time delta
    
    return current_time+1
    

In [33]:
def get_possible_keys(context):
    
    
    keys = []
    
    
    for i in range(len(context)):
        stop = len(context)-i-1
        #for j in range(stop):
        if stop>=1:
            key = '-'.join([str(context[j]) for j in range(stop)])
        
        keys.append(key)
    keys.append('{}-mean'.format(context[0]))
    return keys

In [76]:
def get_steps_no_action(context):
    
 
    new_context = modify_context_no_dosage(context)
    
    context_key = '-'.join([str(c) for c in new_context])
    possible_keys = get_possible_keys(new_context)
    keys = [context_key]
    keys.extend(possible_keys)
    #print(keys)
    #keys = [k+'-{}'.format(action) for k in keys]
    #print(keys)
    i=0
    while keys[i] not in dists:
        #print(i)
        i=i+1
    #print(keys[i])
    #print(keys[-1])
    dist = dists[keys[i]]
    
    
    x = np.random.normal(loc=dist[0],scale=dist[1])
    while(x<0):
         x = np.random.normal(loc=dist[0],scale=dist[1])
    return x
    

In [35]:
def get_steps_action(context,action):
    #nkey = '{}-{}'.format(action,context)

    
    
    this_context = [action]
    this_context.extend(context)
    possible_keys = get_possible_keys(this_context)
    
    context_key = '-'.join([str(c) for c in this_context])
    
    keys = [context_key]
    keys.extend(possible_keys)
    #print(keys)
    #keys = [k+'-{}'.format(action) for k in keys]
    #print(keys)
    i=0
    while keys[i] not in intervention_dists:
        #print(i)
        i=i+1
    #print(keys[i])
    #print(keys[-1])
    dist = intervention_dists[keys[i]]
    
    
    x = np.random.normal(loc=dist[0],scale=dist[1])
    while(x<0):
         x = np.random.normal(loc=dist[0],scale=dist[1])
    return x
    

In [75]:
def modify_context_no_dosage(context):
    
    lkeys = ['group_id','day_of_week','time_of_day','dosage','location','weather','pretreatment','yesterday','variation']
    
    kl = {i:lkeys[i] for i in range(len(lkeys))}
    
    new_context = [context[i] for i,v in kl.items() if v!='dosage']
    
    return new_context

In [36]:
def get_steps(context,action):
    if action==-1:
        return get_steps_no_action(context)

    return get_steps_action(context,action)

In [37]:
def get_next_location(context):
    
    with open('{}location_conditiononed_on_last_location.pkl'.format(root),'rb') as f:
        loc_dists =pickle.load(f)
    
    context_key = '-'.join([str(c) for c in context])
    possible_keys = get_possible_keys(context)
    
    keys = [context_key]
    keys.extend(possible_keys)
    #print(possible_keys)
    i=0
    #print(keys[-1])
    while keys[i] not in loc_dists and i<len(keys):
        i=i+1
    dist = loc_dists[keys[i]]
    
    val = np.argmax(np.random.multinomial(100,dist))
    
    return val
            
                
    
    
    

In [84]:
def get_next_weather(context):
    
    with open('{}weather_conditiononed_on_last_weather_merged.pkl'.format(root),'rb') as f:
        loc_dists =pickle.load(f)
    

    
    relevant_context = [context[get_index('time_of_day')],context[get_index('weather')]]
    
    context_key = '-'.join([str(c) for c in relevant_context])
    possible_keys = get_possible_keys(relevant_context)
    
    keys = [context_key]
    keys.extend(possible_keys)
    #print(keys)
    i=0
    #print(keys[-1])
    while keys[i] not in loc_dists and i<len(keys):
        i=i+1
    dist = loc_dists[keys[i]]
    
    val = np.argmax(np.random.multinomial(100,dist))
    
    return val

In [39]:
#def get_pretreatment(some_steps):
    ##pull in distribution threshold lookup thing
#    return math.log(some_steps+.5)
    

In [40]:
def get_pretreatment(steps):
    chunks =  [[0, 117.],[ 117.,330.],[330.,759.8],[759.8,100000000]]
    
    for i in range(len(chunks)):
        if steps>=chunks[i][0] and steps<chunks[i][1]:
            return i

In [41]:
##what do I need here
def get_new_dosage(current_dosage,action):
    if action==1:
        current_dosage = current_dosage+2
    else:
        current_dosage=current_dosage-1
    if current_dosage>100:
        current_dosage=100
    if current_dosage<1:
        current_dosage=1 
    return current_dosage

In [53]:
def get_context_revised(current_index,current_context,current_steps,decision_time,ysc,variation,last_action):
        
    day_of_week = get_time_of_day(current_index)
    time_of_day = get_day_of_week(current_index)
    
    
    new_previous_step_count = get_pretreatment(current_steps)
    
    
    if decision_time:
        location = get_next_location([current_context[get_index('group_id')],day_of_week,time_of_day,current_context[get_index('location')]])
    
        dosage = get_new_dosage(current_context[get_index('dosage')],last_action)
        
        weather = get_next_weather(current_context)
        
        pretreatment_new = get_pretreatment(current_steps)
        
        
    else:
        location = current_context[get_index('location')]
        dosage = current_context[get_index('dosage')]
        weather = current_context[get_index('weather')]
        pretreatment_new = get_pretreatment(current_steps)
        
        
    return [current_context[0],day_of_week,time_of_day,dosage,location,weather,pretreatment_new,ysc,variation]
    
    

In [43]:
def get_new_lsc(step_slice):
    #print('hi there')
    s =sum(step_slice)**.5
    if s<0:
        return 0
    if s>203:
        return 203
    return s

In [44]:
def get_variation_pre_week(variation,all_steps,time_indices,i):
    
    two_days = time_indices[0].date()+pd.DateOffset(days=1)
    is_set = False
    #first_index_last_day=-1
    if i>two_days:
        #print(i)
        for j in range(len(time_indices)):
            if time_indices[j].date()==i.date():
                if not is_set:
                    #print('set')
                    #print(j)
                    first_index_last_day = j
                    #print(first_index_last_day)
                    is_set = True
            if time_indices[j]== i:
                last_index_last_day = j
    
        pre_steps = all_steps[:first_index_last_day]
        post_steps = all_steps[first_index_last_day:last_index_last_day]
        
        #print(pre_steps)
        #print(post_steps)
        
        return int(np.array(pre_steps).std()>np.array(post_steps).std())
        
    else:
        return variation
    
    

In [45]:
#will be algorithm, needs to communicate with algorithm
def get_action(initial_context,steps):
    return 1

In [64]:
def simulate_run(num_people,time_indices,decision_times):
    
    
    initial_context = get_initial_context(num_people,time_indices[0])
    
    
    initial_context = initial_context[0]
    
    
    
    
    initial_steps = get_steps(initial_context,0)
    current_steps = initial_steps
    action = -1 
    all_steps = []
    
    last_day = time_indices[0]
    
    new_day = False
    
    #for d in range(num_days):
    
    
    start_of_day = 0 
    end_of_day=0
    current_index=0
    
    
    first_week = time_indices[0].date()+pd.DateOffset(days=7)
    
    for i in time_indices:
        
        if i.date()!=last_day.date():
            #print('trigger')
            #print(i.date())
            #print(last_day.date())
            #print('hi there')
            new_day=True
            
            
            
        decision_time = bool(i in decision_times)
        #print(decision_time)
        if i!=time_indices[0]:
            #decision_time = bool(i in decision_times)
            
            ##need to modify this
            #my_context = get_context(initial_context,current_steps,i,decision_time)
            lsc = initial_context[get_index('yesterday')]
            variation = initial_context[get_index('variation')]
            if new_day:
                #lsc=0
                
                
                ##would love to break this out more cleanly 
                if i<first_week:
                    variation = get_variation_pre_week(variation,all_steps,time_indices,last_day)
                else:
                    variation = get_variation(all_steps,time_indices,last_day)
                
                
                lsc = get_new_lsc(all_steps[start_of_day:end_of_day])
                #variation = get_new_variation()
                
            
            ##action will be the last action
            my_context = get_context_revised(i,initial_context,current_steps,decision_time,lsc,variation,action)
            #return my_context
            if i in decision_times:
                print('decision time')
                action = get_action(my_context,current_steps)
                #print(action)
            else:
                action = -1
            ##redo get_steps
            next_steps = get_steps(my_context,action) 
            all_steps.append(next_steps)
            initial_context = my_context
            current_steps = next_steps
        else:
            if i in decision_times:
                print('decision time')
                action = get_action(initial_context,current_steps)
            else:
                action = -1
            next_steps = get_steps(initial_context,action) 
            all_steps.append(next_steps)
            current_steps = next_steps
        if new_day:
            
            start_of_day = current_index
            new_day=False
        last_day = i
        end_of_day = current_index  
        current_index = current_index+1
            
    return all_steps

A study needs some days. So you can specify days like this: 


In [47]:
days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=90),freq='30T')

I made 90 days because that's  the length of the study
However, because I am lazy, I'm only testing for a week

In [67]:
temp_indices = days[:24*7]
temp_indices

DatetimeIndex(['2015-07-22 00:00:00', '2015-07-22 00:30:00',
               '2015-07-22 01:00:00', '2015-07-22 01:30:00',
               '2015-07-22 02:00:00', '2015-07-22 02:30:00',
               '2015-07-22 03:00:00', '2015-07-22 03:30:00',
               '2015-07-22 04:00:00', '2015-07-22 04:30:00',
               ...
               '2015-07-25 07:00:00', '2015-07-25 07:30:00',
               '2015-07-25 08:00:00', '2015-07-25 08:30:00',
               '2015-07-25 09:00:00', '2015-07-25 09:30:00',
               '2015-07-25 10:00:00', '2015-07-25 10:30:00',
               '2015-07-25 11:00:00', '2015-07-25 11:30:00'],
              dtype='datetime64[ns]', length=168, freq='30T')

You can pass in a bunch of decision times. I just tested with one. 

In [72]:
decision_times = pd.DatetimeIndex(['2015-07-25 11:00:00'])
num_people = 2

Right now I think the number of people is being totally ignored. This needs to be fixed. 

In [83]:
x=simulate_run(num_people,temp_indices,decision_times)

['1-1', '1', '1', '1-mean']
decision time


In [62]:
np.array(x).mean()
#np.array(x).std()

215.64595046138658

In [63]:
np.array(x).mean()
np.array(x).std()

278.53952167963189

In [95]:
all_runs = []
for i in range(5):
    all_runs.append(simulate_run(num_people,temp_indices,decision_times))

In [96]:
all_runs

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1530.6373573890496,
  476.8428443879265,
  463.8945242532511,
  36.88658497265851,
  1093.9564008314815,
  113.97741330889428,
  376.53717527044284,
  513.3074950103468,
  490.6536310136565,
  1017.3541702260319,
  790.3468987381691,
  716.333280805501,
  414.72251853170985,
  63.52155833891641,
  1566.4005028245874,
  1388.476137638731,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  636.6805985732924,
  820.0060802991552,
  770.6308350317217,
  504.59359670018705,
  577.9150913132742,
  1166.6431503705087,
  722.2596270620205,
  583.861251708571,
  973.4166263150449,
  734.4477147694403,
  115.76340272879986,
  542.9286189832601,
  677.1204756868657,
  1025.1098413915547,
  1263.4872393696246,
  490.6205144987687,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  378.42294599679917,
  222.66418643832725,
  67.43486682914849,
  114.01199711777178,
  151.09049384810874,
  228.01141131295043,
  1030.151015168952

In [80]:
    with open('{}weather_conditiononed_on_last_weather.pkl'.format(root),'rb') as f:
        loc_dists =pickle.load(f)

In [81]:
loc_dists.keys()

dict_keys(['0-3', '0-1', '1-3', '0-4', '0-2', '1-4', '1-2'])